In [122]:
import argparse
import os 
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [124]:
import shutil
shutil.rmtree("images") # 删除文件夹

# 保存生成的图片
os.makedirs("images", exist_ok=True)

In [126]:
# 定义参数
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=1, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=512, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--n_classes", type=int, default=10, help="number of classes for dataset")
parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval between image sampling")
opt = parser.parse_args(args=[])
print(opt)


Namespace(b1=0.5, b2=0.999, batch_size=512, channels=1, img_size=32, latent_dim=100, lr=0.0002, n_classes=10, n_cpu=8, n_epochs=1, sample_interval=400)


In [127]:
img_shape = (opt.channels, opt.img_size, opt.img_size)
print(img_shape)

(1, 32, 32)


In [128]:
cuda = True if torch.cuda.is_available() else False

In [129]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normailize = True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normailize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normailize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))), # Return the product of array elements over a given axis.
            nn.Tanh()
        )

    def forward(self, nosie, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), nosie), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img
            

In [130]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity


In [131]:
# loss function 
adversarial_loss = torch.nn.MSELoss()

In [132]:
# initialize 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator().to(device)
discriminator = Discriminator().to(device)

from torchsummary import summary
# summary(generator, img_shape)
# summary(discriminator, input_size=(img_shape))
# print(generator)
# print(discriminator)

In [133]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [134]:
# data loader 
# os.makedirs("/data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data/",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

In [135]:
# optimizers 
optimizers_G = torch.optim.Adam(generator.parameters(), lr = opt.lr, betas=(opt.b1, opt.b2))
optimizers_D = torch.optim.Adam(discriminator.parameters(), lr = opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [136]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


In [137]:
# training 

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad = False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad = False)

        # configure input 
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # train Generator

        optimizers_G.zero_grad()

        # sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizers_G.step()

        # train Discriminator

        optimizers_D.zero_grad()

        # loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # loss for fake images 
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizers_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

[Epoch 0/1] [Batch 0/118] [D loss: 0.441419] [G loss: 0.981130]
[Epoch 0/1] [Batch 1/118] [D loss: 0.220375] [G loss: 0.955597]
[Epoch 0/1] [Batch 2/118] [D loss: 0.063530] [G loss: 0.931369]
[Epoch 0/1] [Batch 3/118] [D loss: 0.014326] [G loss: 0.905540]
[Epoch 0/1] [Batch 4/118] [D loss: 0.035425] [G loss: 0.890934]
[Epoch 0/1] [Batch 5/118] [D loss: 0.013000] [G loss: 0.888509]
[Epoch 0/1] [Batch 6/118] [D loss: 0.014019] [G loss: 0.884337]
[Epoch 0/1] [Batch 7/118] [D loss: 0.014125] [G loss: 0.879548]
[Epoch 0/1] [Batch 8/118] [D loss: 0.011374] [G loss: 0.865650]
[Epoch 0/1] [Batch 9/118] [D loss: 0.012668] [G loss: 0.846047]
[Epoch 0/1] [Batch 10/118] [D loss: 0.011982] [G loss: 0.832983]
[Epoch 0/1] [Batch 11/118] [D loss: 0.012652] [G loss: 0.822902]
[Epoch 0/1] [Batch 12/118] [D loss: 0.013515] [G loss: 0.807994]
[Epoch 0/1] [Batch 13/118] [D loss: 0.014611] [G loss: 0.787151]
[Epoch 0/1] [Batch 14/118] [D loss: 0.014854] [G loss: 0.775464]
[Epoch 0/1] [Batch 15/118] [D loss:

In [138]:
input = torch.LongTensor([[1,2,4, 5], [4, 3, 2, 9]])
print(input)
embedding = nn.Embedding(10, 3)
print(embedding)
embedding(input).shape 

tensor([[1, 2, 4, 5],
        [4, 3, 2, 9]])
Embedding(10, 3)


torch.Size([2, 4, 3])

In [139]:
m = nn.BatchNorm1d(100)
input = torch.randn(20, 100)
print(input.shape)
output = m(input)
print(output.shape)

torch.Size([20, 100])
torch.Size([20, 100])
